In [4]:
!pip install import-ipynb

In [5]:
import argparse
from math import log10
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from types import SimpleNamespace

import import_ipynb
import data
import model

In [ ]:
# Zoom faktörüne gerek kalmadı
args = SimpleNamespace(nb_epochs=10, cuda=True)

device = torch.device("cuda:0" if (torch.cuda.is_available() and args.cuda) else "cpu")
torch.manual_seed(0)
torch.cuda.manual_seed(0)

# Parameters
BATCH_SIZE = 16
NUM_WORKERS = 0  # Windows için 0

# Dataset ve DataLoader
%run data.ipynb

trainset = DatasetFromFolder(
    low_res_dir=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\train\Data",
    high_res_dir=  r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\train\Labels"
)

testset = DatasetFromFolder(
    low_res_dir=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\test\Data",
    high_res_dir=r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\test\Labels"
)

trainloader = DataLoader(dataset=trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
testloader = DataLoader(dataset=testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Model, Loss ve Optimizer
%run model.ipynb  # model.ipynb içeriğini .py dosyasına taşıman daha sağlıklı olur

model = SRCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam([
    {"params": model.conv1.parameters(), "lr": 0.0001},
    {"params": model.conv2.parameters(), "lr": 0.0001},
    {"params": model.conv3.parameters(), "lr": 0.00001},
], lr=0.00001)

# ✅ Eğitim döngüsü
for epoch in range(args.nb_epochs):
    model.train()
    epoch_loss = 0
    for input, target in trainloader:
        input, target = input.to(device), target.to(device)
        optimizer.zero_grad()
        out = model(input)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(trainloader)
    print(f"📘 Epoch {epoch+1}/{args.nb_epochs} - Training Loss: {avg_loss:.6f}")

    # ✅ Test & PSNR Hesaplama
    model.eval()
    avg_psnr = 0
    with torch.no_grad():
        for input, target in testloader:
            input, target = input.to(device), target.to(device)
            out = model(input)
            loss = criterion(out, target)
            psnr = 10 * log10(1 / loss.item())
            avg_psnr += psnr

    avg_psnr /= len(testloader)
    print(f"🔍 Epoch {epoch+1} - Avg PSNR: {avg_psnr:.2f} dB")

    # ✅ Modeli Kaydet
    save_path = f"model_{epoch+1}.pth"
    torch.save(model, save_path)
    print(f"💾 Model kaydedildi: {save_path}")


📘 Epoch 1/10 - Training Loss: 0.249080
🔍 Epoch 1 - Avg PSNR: 7.56 dB
💾 Model kaydedildi: model_1.pth
📘 Epoch 2/10 - Training Loss: 0.101115
🔍 Epoch 2 - Avg PSNR: 15.35 dB
💾 Model kaydedildi: model_2.pth
📘 Epoch 3/10 - Training Loss: 0.012612
🔍 Epoch 3 - Avg PSNR: 20.46 dB
💾 Model kaydedildi: model_3.pth
📘 Epoch 4/10 - Training Loss: 0.008416
🔍 Epoch 4 - Avg PSNR: 21.08 dB
💾 Model kaydedildi: model_4.pth
📘 Epoch 5/10 - Training Loss: 0.007425
🔍 Epoch 5 - Avg PSNR: 21.64 dB
💾 Model kaydedildi: model_5.pth
📘 Epoch 6/10 - Training Loss: 0.006441
🔍 Epoch 6 - Avg PSNR: 22.29 dB
💾 Model kaydedildi: model_6.pth
📘 Epoch 7/10 - Training Loss: 0.005533
🔍 Epoch 7 - Avg PSNR: 22.96 dB
💾 Model kaydedildi: model_7.pth
📘 Epoch 8/10 - Training Loss: 0.004750
🔍 Epoch 8 - Avg PSNR: 23.58 dB
💾 Model kaydedildi: model_8.pth
📘 Epoch 9/10 - Training Loss: 0.004178
🔍 Epoch 9 - Avg PSNR: 24.09 dB
💾 Model kaydedildi: model_9.pth
📘 Epoch 10/10 - Training Loss: 0.003761
🔍 Epoch 10 - Avg PSNR: 24.48 dB
💾 Model kay